In [1]:
import pickle
import os

In [2]:
# with open(os.path.join("data","yelp_2017","dict.pkl"),"rb") as f:
#     word_to_ind = pickle.load(f,encoding="bytes")
# with open(os.path.join("data","yelp_2017","word_emb.pkl"),"rb") as f:
#     emb = pickle.load(f,encoding="bytes")

In [3]:
# max(word_to_ind.values()),emb.shape

In [4]:
import sqlite3

conn = sqlite3.connect('yelp_dataset.db')
c = conn.cursor()

def Select(q):
    return c.execute(q)

def Update(q):
    c.execute(q)
    conn.commit()
    
        
# conn.close()

In [5]:
# x=Update("create view RevAB_CA as select \
# User.id as user_id, User.review_count as user_review_count, User.average_rating as user_rating, \
# Business.city as city, Business.state as state, Business.ratings as item_rating, Business.id as item_id, \
# Reviews.id as review_id, Reviews.ts as ts, Reviews.rating as review_rating \
# from Reviews \
# Inner JOIN User on User.id = Reviews.user_id \
# INNER JOIN Business on Business.id = Reviews.business_id \
# where Business.city = \"Las Vegas\"")

In [6]:
data = Select("Select * from RevAB_CA where user_review_count > 15 LIMIT 10000")
#

In [7]:
query = data
data = []
for row in query:
    data += [row]

In [8]:
items = list(set([x[6] for x in data]))
users = list(set([x[0] for x in data]))
# items_ind = {items[i]:i for i in range(len(items))}

# users_ind = {users[i]:i for i in range(len(users))}

In [9]:
len(items),len(users),len(data[0])

(1015, 5661, 11)

In [10]:
from collections import defaultdict 
rev_u = defaultdict(lambda: defaultdict(dict))
rev_i = defaultdict(lambda: defaultdict(dict))
for review in data:
    user = review[0]
    item = review[6]
    text = review[10]#o
    rev_u [user][item] = text
    rev_i [item][user] = text
rev_u = dict(rev_u)
rev_i = dict(rev_i)

In [95]:
# words = set(word_to_ind.keys())
# wordList = list(words)

In [11]:
import difflib
import numpy as np
filename = 'glove.6B.50d.txt'
def loadGloVe(filename):
    vocab = set()
    embd = {}
    file = open(filename,'r')
    word = None
    with open(filename,encoding="utf-8") as f:
        try:
            for line in f:
                row = line.strip().split(' ')
                word = row[0]
                vocab.add(row[0])
                embd[word] = np.array(row[1:])
            
        except Exception as e:
            print(word)
    print('Loaded GloVe!')
    file.close()
    return vocab,embd
vocab_set,embd = loadGloVe(filename)
# vocab_size = len(vocab)
# vocab_set = set(vocab)
# embedding_dim = len(embd[0])
# embedding = np.asarray(embd)


                    

Loaded GloVe!


In [22]:
vocab[:5],vocab_size

(['the', ',', '.', 'of', 'to'], 400000)

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dense,Dropout,BatchNormalization, SimpleRNN, GRU, LSTM,Bidirectional,Conv2D,Merge,Flatten,MaxPooling2D

Using TensorFlow backend.


In [13]:
def deepCONN(user_shape,item_shape):
    model1 = Sequential()
    model1.add (Conv2D(32, (2,user_shape[1]), padding='same',
                 input_shape=user_shape, activation="relu"))
    model1.add(MaxPooling2D(pool_size=(user_shape[0],1)))     
    model1.add(Flatten())
    model1.add(Dense(100,activation="relu"))
    model1.add(BatchNormalization())
                     
                     
    model2 = Sequential()
    model2.add (Conv2D(32, (2,item_shape[1]), padding='same',
                 input_shape=item_shape, activation="relu"))
    model2.add(MaxPooling2D(pool_size=(item_shape[0],1)))
    model2.add(Flatten())
    model2.add(Dense(100,activation="relu"))
    model2.add(BatchNormalization())
                     
    model = Sequential()
    model.add(Merge([model1,model2],mode='concat'))
    model.add(Dense(32,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    return model
    

In [14]:
max_len = 40

In [15]:
model = deepCONN((max_len,50,1),(max_len,50,1))

c:\python3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [49]:
# import numpy 
# def embedding(words):
#     embed = []
#     for word in words:
#         embed += [embd[word]]
#     for _ in range(100-len(words)):
#         embed += [[0]*50]
#     return numpy.array(embed)

In [63]:
import numpy
def embedding(text,maxlen):
    text = text.split()
    count = 0
    _w = []
    for word in text:
        word = word.lower()
        if word in vocab_set:
            _w += [embd[word]]
        elif word[:-1] in vocab_set and word[-1] == ".":
            _w += [embd[word[:-1]]]
            if len(_w) == max_len:
                break
            _w += [embd[word[-1]]]
        else:
            count += 1
        if len(_w) == max_len:
            break
#     print(len(_w))
    for _ in range(maxlen-len(_w)):
        _w += [numpy.array([0]*50)]
    ar= numpy.array(_w)
    ar = numpy.reshape(ar,(maxlen,50,1))
    return ar

In [64]:
user_emb, item_emb = {},{}
for user in rev_u:
    user_emb[user] = embedding(" ".join([rev_u[user][item] for item in rev_u[user] ]),max_len)
for item in rev_i:
    item_emb[item] = embedding(" ".join([rev_i[item][user] for user in rev_i[item]]),max_len)

In [65]:
# for user in rev_u:
#     user_emb[user] = embedding(user_emb[user])
# for item in rev_i:
#     item_emb[item] = embedding(item_emb[item])

In [68]:
X_user = []
X_item = []
Y = []
for i in range(5000):
    user,item,rating = data[i][0],data[i][6],data[i][9]
#     print(embedding(data[i][10],max_len).shape)
    X_user += [user_emb[user]]
    X_item += [item_emb[item]]
#     print(user_emb[user].shape,item_emb[item].shape)
    Y += [rating]
X_user = numpy.array(X_user)
X_item = numpy.array(X_item)
Y = numpy.array(Y)
# embd["and"].shape
X_user.shape
X_item.shape

(5000, 40, 50, 1)

In [69]:
model.compile(loss="mean_squared_error",optimizer=Adam(lr=0.004),metrics=["accuracy"])
model.fit([X_user,X_item],Y)

Epoch 1/10
5000/5000 [==============================] - 18s - loss: 2.2789 - acc: 0.2250    
Epoch 2/10
5000/5000 [==============================] - 17s - loss: 1.8472 - acc: 0.2518    
Epoch 3/10
5000/5000 [==============================] - 17s - loss: 1.7025 - acc: 0.2672    
Epoch 4/10
5000/5000 [==============================] - 18s - loss: 1.5582 - acc: 0.2826    
Epoch 5/10
5000/5000 [==============================] - 17s - loss: 1.3990 - acc: 0.3004    
Epoch 6/10
5000/5000 [==============================] - 18s - loss: 1.3422 - acc: 0.3114    
Epoch 7/10
5000/5000 [==============================] - 18s - loss: 1.2290 - acc: 0.3362    
Epoch 8/10
5000/5000 [==============================] - 18s - loss: 1.1804 - acc: 0.3278    
Epoch 9/10
5000/5000 [==============================] - 18s - loss: 1.1026 - acc: 0.3430    
Epoch 10/10
5000/5000 [==============================] - 18s - loss: 1.0639 - acc: 0.3572    
